# Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

## Домашнее задание

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, GlobalMaxPool2D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

### Загрузка и подготовка данных

In [2]:
df = pd.read_excel('/content/drive/MyDrive/отзывы за лето.xlsx')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
df = df.dropna()

In [4]:
df.loc[:, 'Content'] = df['Content'].astype(str)

In [5]:
X = df['Content']
y = df['Rating']

In [6]:
y = np.where(y == 5, 1, 0)

In [7]:
num_classes = 1
epochs = 20
batch_size = 512

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
text_corpus_train = X_train
text_corpus_test = X_test

In [10]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [12]:
max_words = word_count
max_len = training_length

### Построение CNN сети

In [13]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
30/30 [==============================] - 25s 803ms/step - loss: 0.5646 - accuracy: 0.7163 - val_loss: 0.4215 - val_accuracy: 0.8421
Epoch 2/20
30/30 [==============================] - 22s 727ms/step - loss: 0.3604 - accuracy: 0.8648 - val_loss: 0.3210 - val_accuracy: 0.8845


In [16]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

              precision    recall  f1-score   support

           0       0.80      0.73      0.76      1204
           1       0.89      0.93      0.91      2928

    accuracy                           0.87      4132
   macro avg       0.85      0.83      0.83      4132
weighted avg       0.86      0.87      0.87      4132



### Построение RNN

In [17]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [18]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [19]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
30/30 [==============================] - 12s 331ms/step - loss: 0.5467 - accuracy: 0.7381 - val_loss: 0.4340 - val_accuracy: 0.8234
Epoch 2/20
30/30 [==============================] - 9s 306ms/step - loss: 0.4284 - accuracy: 0.8243 - val_loss: 0.3767 - val_accuracy: 0.8500


In [20]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

              precision    recall  f1-score   support

           0       0.72      0.66      0.69      1204
           1       0.86      0.90      0.88      2928

    accuracy                           0.83      4132
   macro avg       0.79      0.78      0.79      4132
weighted avg       0.82      0.83      0.82      4132



### Построение CNN+RNN

In [21]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 conv1d_1 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 3,209,365
Trainable params: 3,209,365
Non-trainable params: 0
____________________________________________

In [22]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 176, 30)           740340    
                                                                 
 masking_1 (Masking)         (None, 176, 30)           0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                6080      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 750,645
Trainable params: 750,645
Non-tr

In [23]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 masking_2 (Masking)         (None, 176, 128)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 33s 1s/step - loss: 0.5301 - accuracy: 0.7496 - val_loss: 0.4081 - val_accuracy: 0.8367
Epoch 2/20
30/30 [==============================] - 32s 1s/step - loss: 0.3864 - accuracy: 0.8519 - val_loss: 0.3352 - val_accuracy: 0.8760
              precision    recall  f1-score   support

           0       0.81      0.67      0.73      1204
           1       0.87      0.93      0.90      2928

    accuracy                           0.86      4132
   macro avg       0.84      0.80      0.82      4132
weighted avg       0.85      0.86      0.85      4132



### Построение CNN+MaxPool+RNN

In [26]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True)) # keepdims Сохраняет размерность для SimpleRNN
model.add(SimpleRNN(64))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 masking_4 (Masking)         (None, 176, 128)          0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 176, 128)          32896     
                                                                 
 conv1d_4 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 1, 128)           0         
 balMaxPooling1D)                                                
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 64)                12352     
                                                      

In [27]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 52s 2s/step - loss: 0.5101 - accuracy: 0.7714 - val_loss: 0.4131 - val_accuracy: 0.8294
Epoch 2/20
30/30 [==============================] - 48s 2s/step - loss: 0.3972 - accuracy: 0.8493 - val_loss: 0.3428 - val_accuracy: 0.8724
              precision    recall  f1-score   support

           0       0.82      0.66      0.73      1204
           1       0.87      0.94      0.90      2928

    accuracy                           0.86      4132
   macro avg       0.85      0.80      0.82      4132
weighted avg       0.86      0.86      0.85      4132



### Построение RNN+CNN+RNN

In [28]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3, activation='relu'))
model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(64, 3, activation='relu'))
model.add(SimpleRNN(64, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(32, 2, activation='relu'))
model.add(SimpleRNN(32, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 masking_5 (Masking)         (None, 176, 128)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 174, 128)          32896     
                                                                 
 conv1d_6 (Conv1D)           (None, 172, 64)           24640     
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 172, 64)           8256      
                                                                 
 conv1d_7 (Conv1D)           (None, 171, 32)          

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 66s 2s/step - loss: 0.6104 - accuracy: 0.6874 - val_loss: 0.4947 - val_accuracy: 0.7623
Epoch 2/20
30/30 [==============================] - 64s 2s/step - loss: 0.4846 - accuracy: 0.8038 - val_loss: 0.3862 - val_accuracy: 0.8560
              precision    recall  f1-score   support

           0       0.73      0.66      0.70      1204
           1       0.87      0.90      0.88      2928

    accuracy                           0.83      4132
   macro avg       0.80      0.78      0.79      4132
weighted avg       0.83      0.83      0.83      4132



### Построение CNN+GRU

In [30]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(32, 4, 8, activation='relu'))
model.add(Dropout(0.5))
model.add(GlobalMaxPool1D(keepdims=True))
model.add(LSTM(32))

model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 8s 167ms/step - loss: 0.6443 - accuracy: 0.6897 - val_loss: 0.5974 - val_accuracy: 0.7157
Epoch 2/20
30/30 [==============================] - 4s 142ms/step - loss: 0.5361 - accuracy: 0.7443 - val_loss: 0.4796 - val_accuracy: 0.8355
              precision    recall  f1-score   support

           0       0.68      0.60      0.64      1204
           1       0.84      0.88      0.86      2928

    accuracy                           0.80      4132
   macro avg       0.76      0.74      0.75      4132
weighted avg       0.80      0.80      0.80      4132

